# 2. Classify Orientations 
## Given images are rotated and we need to classify the direction in which they are facing
## To do so, we are going to use the ResNet18 

In [1]:
import torch
import torchvision
from torch.utils.data import Dataset
from torchvision.io import read_image
from torchvision.utils import save_image
from torchvision.transforms import ToTensor, Compose, Resize, Grayscale, Normalize, Lambda
import os
import torchvision.transforms as T
import pydicom
import matplotlib.pyplot as plt
import pandas as pd
from torch.utils.data import DataLoader
from torch import optim
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Define a custom class for dataset 
class CustomDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        image = T.ToPILImage() (image)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [3]:
device = torch.device('cuda')

In [4]:
# Defining the transform for loading the data
transform = Compose([
    ToTensor(),
    Lambda(lambda x: x.repeat(3,1,1)),
    Normalize((0.5,), (0.5,))
])

# Define the training image directory and annotations csv
directions01_train_img_dir = '/kaggle/input/chestxraydirections01/dataset/Directions01'
directions01_train_annotations_file = '/kaggle/input/chestxraydirections01/dataset/Directions01/list_train.txt'

# Defining the training dataset and data loader
directions_train_dataset = CustomDataset(
    annotations_file=directions01_train_annotations_file,
    img_dir=directions01_train_img_dir,
    transform=transform,
)
directions_train_dataloader = DataLoader(directions_train_dataset, batch_size=64, shuffle=True)

# Define the testing image directory and annotations csv
directions01_test_img_dir = '/kaggle/input/chestxraydirections01/dataset/Directions01'
directions01_test_annotations_file = '/kaggle/input/chestxraydirections01/dataset/Directions01/list_test.txt'

# Defining the testing dataset and data loader
directions_test_dataset = CustomDataset(
    annotations_file=directions01_test_annotations_file,
    img_dir=directions01_test_img_dir,
    transform=transform
)
directions_test_loader = DataLoader(directions_test_dataset, batch_size=64, shuffle=True)

In [5]:
# Definining the resnet18 model
from torchvision.models import resnet18, ResNet18_Weights
import torch.nn as nn
class Resnet18Classifier(nn.Module):
    def __init__(self, num_classes):
        super(Resnet18Classifier, self).__init__()
        self.resnet18 = resnet18(pretrained=False).eval()
        num_ftrs = self.resnet18.fc.in_features
        self.resnet18.fc = nn.Linear(num_ftrs, num_classes)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.resnet18(x)

In [6]:
# Initializing the resnet18 classifier with number of output models
num_classes = 4
resnet_18_classifier = Resnet18Classifier(num_classes).to(device)

# Defining the loss_function and the optimizer
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet_18_classifier.parameters(), lr = 0.001)
resnet_18_classifier

Resnet18Classifier(
  (resnet18): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=Tru

In [7]:
# Defining the label mapping for each direction
label_mapping = {
    'left': 0,
    'right': 1,
    'up': 2,
    'down': 3
}

In [8]:
# Training the model
num_epochs = 10
for epoch in range(num_epochs):
    for images, labels in directions_train_dataloader:
        numeric_labels = [label_mapping[label] for label in labels]
        numeric_labels_tensor = torch.tensor(numeric_labels)
        images, labels = images.to(device), numeric_labels_tensor.to(device)
        optimizer.zero_grad()
        outputs = resnet_18_classifier(images)
#         print(f'Output is {outputs}')
#         print(f'Labels are {labels}')
        loss = loss_func(outputs, labels)
        loss.backward()
        optimizer.step()
        
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

print("Training finished")

Epoch [1/10], Loss: 1.7017
Epoch [2/10], Loss: 1.6169
Epoch [3/10], Loss: 1.2916
Epoch [4/10], Loss: 0.0690
Epoch [5/10], Loss: 0.1052
Epoch [6/10], Loss: 0.0010
Epoch [7/10], Loss: 0.2102
Epoch [8/10], Loss: 0.0293
Epoch [9/10], Loss: 0.0002
Epoch [10/10], Loss: 0.0004
Training finished


In [9]:
# Testing the model
correct = 0
total = 0
# Don't calculate gradients during evaluation
with torch.no_grad():
    for images, labels in directions_test_loader:
        numeric_labels = [label_mapping[label] for label in labels]
        numeric_labels_tensor = torch.tensor(numeric_labels)
        images, labels = images.to(device), numeric_labels_tensor.to(device)
        outputs = resnet_18_classifier(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        # print(f'Run for instance {total} and {correct}')

accuracy = 100 * correct / total
print(f"Test Accuracy for directions: {accuracy:.2f}%")

Test Accuracy for directions: 100.00%


# Achieved 100% accuracy with Resnet50 for classifying image directions.